# Лабораторная работа 5



In [ ]:
import sys
import numpy as np

In [ ]:
class Simplex:
    eps = 1 / 10**8

    def __init__(self, source):
        self.n = len(source)
        self.m = len(source[0])
        opt_func = source[self.n - 1]
        
        self.create_table_with_synthetic_basis(source)
        self.calculate()

        all_null = True
        for j in range(self.m - self.n + 1):
            if self.table[self.n - 1][j] > Simplex.eps:
                all_null = False
                break
        if not all_null:
            print("В результате получения первого опорного плана не все искусственные переменные равны нулю. ", "Решений нет!")
            sys.exit()

        new_table = [[0] * (self.m - self.n + 1) for i in range(self.n)]
        for i in range(len(new_table)):
            for j in range(len(new_table[0])):
                new_table[i][j] = self.table[i][j]

        for j in range(len(new_table[0])):
            sum_col = 0
            for i in range(len(new_table) - 1):
                sum_col += new_table[i][j] * opt_func[self.basis[i]]
            new_table[len(new_table) - 1][j] = sum_col - opt_func[j]
        new_table[len(new_table) - 1] = list(np.array(new_table[len(new_table) - 1]) * (-1))

        self.table = new_table
        self.n = len(self.table)
        self.m = len(self.table[0])

    def calculate(self):
        while not self.is_end():
            main_col = self.find_main_col()
            main_row = self.find_main_row(main_col)

            if main_row == -1:
                print("Не удалось выбрать опорный элемент. Задача не имеет решений, так как ОДР не ограничена")
                sys.exit()

            self.basis[main_row] = main_col
            self.make_step(main_row, main_col)

        result = [0 for i in range(self.m - 1)]
        for i in range(len(self.basis)):
            if self.basis[i] <= self.m - 1:
                result[self.basis[i] - 1] = self.table[i][0]
        return [self.table, result]

    def is_end(self):
        flag = True
        for j in range(1, self.m):
            if self.table[self.n - 1][j] < -Simplex.eps:
                flag = False
                break
        return flag

    def find_main_col(self):
        main_col = 1
        for j in range(2, self.m):
            if self.table[self.n - 1][j] < self.table[self.n - 1][main_col]:
                main_col = j
        return main_col

    def find_main_row(self, main_col):
        main_row = -1
        for i in range(self.n - 1):
            if self.table[i][main_col] > Simplex.eps:
                main_row = i
                break

        if main_row == -1:
            return -1

        for i in range(main_row + 1, self.n - 1):
            if (self.table[i][main_col] > Simplex.eps) and (self.table[i][0] / self.table[i][main_col] < self.table[main_row][0] / self.table[main_row][main_col]):
                main_row = i

        return main_row

    def make_step(self, main_row, main_col):
        new_table = [[0] * self.m for i in range(self.n)]

        for j in range(self.m):
            new_table[main_row][j] = self.table[main_row][j] / self.table[main_row][main_col]

        for i in range(self.n):
            if i == main_row:
                continue

            for j in range(self.m):
                new_table[i][j] = self.table[i][j] - (self.table[main_row][j] / self.table[main_row][main_col]) * self.table[i][main_col]
                new_table[i][j] = new_table[i][j]

        self.table = new_table

    def create_table_with_synthetic_basis(self, source):
        self.basis = list()
        self.table = [[0] * (self.m + self.n - 1) for i in range(self.n)]

        # create first table
        for i in range(self.n):
            for j in range(len(self.table[0])):
                if j < self.m:
                    self.table[i][j] = source[i][j]
                else:
                    self.table[i][j] = 0

            # add basis
            if (self.m + i) < len(self.table[0]):
                self.table[i][self.m + i] = 1
                self.basis.append(self.m + i)

        self.m = len(self.table[0])

        for j in range(self.m):
            sum = 0
            for i in range(self.n - 1):
                sum -= self.table[i][j]
            self.table[self.n - 1][j] = sum
        for basis_col in self.basis:
            self.table[self.n - 1][basis_col] = 0

In [ ]:
class LinearProgrammingTask:
    def __init__(self, f, restrictions, max_min):
        if not (len(f) + 1 == len(restrictions[0]) and (max_min == "max" or max_min == "min")):
            print("bad task data")
            sys.exit()

        self.f = list(f)
        for i in range(len(f) - 1, 0, -1):
            f[i], f[i - 1] = f[i - 1], f[i]
        
        # reversing
        if max_min == "max":
            f = np.array(f) * (-1)

        table = np.array([[0.0] * (len(restrictions[0]) - 1) for i in range(len(restrictions) + 1)])
        table[len(restrictions)] = list(f)
        for i in range(len(restrictions)):
            for j in range(len(restrictions[0])):
                if j < len(restrictions[0]) - 2:
                    table[i][j + 1] = restrictions[i][j]
                elif j == len(restrictions[0]) - 2:
                    table[i][0] = restrictions[i][j + 1]

        # change '>=' to '<='
        for i in range(len(restrictions)):
            if restrictions[i][len(restrictions[0]) - 2] == ">=":
                table[i] = list(np.array(table[i]) * (-1))

        # change '<=' to '='
        for i in range(len(restrictions)):
            if restrictions[i][len(restrictions[0]) - 2] != "=":
                new_column = np.zeros(len(restrictions) + 1)
                new_column[i] = 1
                table = np.column_stack((table, new_column))

        # change b to positive
        for i in range(len(table)):
            if table[i][0] < 0:
                table[i] = list(np.array(table[i]) * (-1))
        self.table = table

    def solve(self):
        simplex = Simplex(source=self.table)
        result_table, result = simplex.calculate()
        end_result = []
        result_f = 0
        for i in range(len(self.f) - 1):
            end_result.append(round(result[i], 4))
            result_f += self.f[i] * end_result[i]
        print("RESULT:")
        print(end_result)
        print("f= " + str(result_f))

## Тестовые задания

In [ ]:
test = LinearProgrammingTask(
    f=[1, 1, 0],
    restrictions=[[3, 5, "<=", 30],
                  [4, -3, "<=", 12],
                  [1, -3, ">=", 6]],
    max_min="max")
test.solve()

В результате получения первого опорного плана не все искусственные переменные равны нулю.  Решений нет!


SystemExit: ignored

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

task1 = LinearProgrammingTask(
    f=[-6, -1, -4, 5, 0],
    restrictions=[[3, 1, -1, 1, "=", 4],
                  [5, 1, 1, -1, "=", 4]],
    max_min="min")
task1.solve()

RESULT:
[0, 4.0, 0.0, 0]
f= -4.0


In [ ]:
task2 = LinearProgrammingTask(
    f=[-1, -2, -3, 1, 0],
    restrictions=[[1, -3, -1, -2, "=", -4],
                  [1, -1, 1, 0, "=", 0]],
    max_min="min")
task2.solve()

RESULT:
[2.0, 2.0, 0, 0]
f= -6.0


In [ ]:
task3 = LinearProgrammingTask(
    f=[-1, -2, -1, 3, -1, 0],
    restrictions=[[1, 1, 0, 2, 1, "=", 5],
                  [1, 1, 1, 3, 2, "=", 9],
                  [0, 1, 1, 2, 1, "=", 6]],
    max_min="min")
task3.solve()

RESULT:
[3.0, 2.0, 4.0, 0, 0]
f= -11.0


In [ ]:
task4 = LinearProgrammingTask(
    f=[-1, -1, -1, 1, -1, 0],
    restrictions=[[1, 1, 2, 0, 0, "=", 4],
                  [0, -2, -2, 1, -1, "=", -6],
                  [1, -1, 6, 1, 1, "=", 12]],
    max_min="min")
task4.solve()

RESULT:
[4.0, 0, 0, 1.0, 7.0]
f= -10.0


In [ ]:
task5 = LinearProgrammingTask(
    f=[-1, 4, -3, 10, 0],
    restrictions=[[1, 1, -1, -10, "=", 0],
                  [1, 14, 10, -10, "=", 11]],
    max_min="min"
)
task5.solve()

RESULT:
[1.0, 0, 1.0, 0]
f= -4.0


In [ ]:
task6 = LinearProgrammingTask(
    f=[-1, 5, 1, -1, 0],
    restrictions=[[1, 3, 3, 1, "<=", 3],
                  [2, 0, 3, -1, "<=", 4]],
    max_min="min"
)
task6.solve()

RESULT:
[2.3333, 0, 0, 0.6667]
f= -3.0


In [ ]:
task7 = LinearProgrammingTask(
    f=[-1, -1, 1, -1, 2, 0],
    restrictions=[[3, 1, 1, 1, -2, "=", 10],
                  [6, 1, 2, 3, -4, "=", 20],
                  [10, 1, 3, 6, -7, "=", 30]],
    max_min="min"
)
task7.solve()

RESULT:
[0.0, 0, 10.0, 0.0, 0]
f= 10.0


## Вопросы на защиту

In [ ]:
test_task_1 = LinearProgrammingTask(
    f=[-3, -2, 0],
    restrictions=[[1, 2, "<=", 7],
                  [2, 1, "<=", 8],
                  [0, 1, "<=", 3]],
    max_min="min"
)
test_task_1.solve()

RESULT:
[3.0, 2.0]
f= -13.0


In [ ]:
# test_task_2 = LinearProgrammingTask(
#     f=[-1, -2, 0],
#     restrictions=[[1, 1, ">=", 1],
#                   [2, -1, ">=", -1],
#                   [1, -2, "<=", 0]],
#     max_min="min"
# )
# test_task_2.solve()

In [ ]:
test_task_3 = LinearProgrammingTask(
    f=[-5, 4, -1, -3, -5, 0],
    restrictions=[[3, -1, 0, 2, 1, "=", 5],
                  [2, -3, 1, 2, 1, "=", 6],
                  [3, -1, 1, 3, 2, "=", 9]],
    max_min="min"
)
test_task_3.solve()

RESULT:
[1.0, 0, 2.0, 0, 2.0]
f= -17.0


In [ ]:
test_task_4 = LinearProgrammingTask(
    f=[-1, 3, 0],
    restrictions=[[1, 2, "<=", 4],
                  [1, -1, ">=", 1],
                  [1, 1, "<=", 8]],
    max_min="max"
)
test_task_4.solve()

RESULT:
[2.0, 1.0]
f= 1.0


# Лабораторная работа 2

## Задание 2

In [ ]:
lab2_task2 = LinearProgrammingTask(
    f=[8, 7, 12, 13, 0],
    restrictions=[[10, 0, 13, 0, "=", 230],
                  [0, 12, 0, 13, "=", 68],
                  [1, 1, 0, 0, "<=", 24],
                  [0, 0, 1, 1, "<=", 24]],
    max_min="min"
)
lab2_task2.solve()

RESULT:
[18.3333, 5.6667, 3.5897, 0]
f= 229.40970000000002


## Задание 3

In [ ]:
lab2_task3 = LinearProgrammingTask(
    f=[12, 8, 0],
    restrictions=[[40, 10, ">=", 1000],
                  [1.25, 2.5, ">=", 100],
                  [2, 1, ">=", 80],
                  [1, 1, ">=", 60]],
    max_min="min"
)
lab2_task3.solve()

RESULT:
[20.0, 40.0]
f= 560.0


## Задание 4

In [ ]:
lab2_task4_1 = LinearProgrammingTask(
    f=[1, 1, 0],
    restrictions=[[4, 2, "<=", 1],
                  [2, 3, "<=", 1]],
    max_min="max"
)
lab2_task4_1.solve()

RESULT:
[0.125, 0.25]
f= 0.375


In [ ]:
lab2_task4_2 = LinearProgrammingTask(
    f=[1, 1, 0],
    restrictions=[[4, 2, ">=", 1],
                  [2, 3, ">=", 1]],
    max_min="min"
)
lab2_task4_2.solve()

RESULT:
[0.125, 0.25]
f= 0.375


## Задание 5

In [ ]:
lab2_task5_1 = LinearProgrammingTask(
    f=[1, 1, 0],
    restrictions=[[8, 4, "<=", 1],
                  [4, 8, "<=", 1],
                  [6, 5, "<=", 1]],
    max_min="max"
)
lab2_task5_1.solve()

RESULT:
[0.0833, 0.0833]
f= 0.1666


In [ ]:
lab2_task5_2 = LinearProgrammingTask(
    f=[1, 1, 1, 0],
    restrictions=[[8, 4, 6, ">=", 1],
                  [4, 8, 5, ">=", 1]],
    max_min="min"
)
lab2_task5_2.solve()

RESULT:
[0.0833, 0.0833, 0]
f= 0.1666


## Задание 6

In [ ]:
lab2_task6_1 = LinearProgrammingTask(
    f=[1, 1, 1, 1, 0],
    restrictions=[[7, 2, 5, 3, "<=", 1],
                  [2, 2, 3, 2, "<=", 1],
                  [5, 3, 4, 1, "<=", 1],
                  [1, 4, 4, 6, "<=", 1]],
    max_min="max"
)
lab2_task6_1.solve()

RESULT:
[0.0769, 0.1978, 0, 0.022]
f= 0.2967


In [ ]:
lab2_task6_2 = LinearProgrammingTask(
    f=[1, 1, 1, 1, 0],
    restrictions=[[7, 2, 5, 1, ">=", 1],
                  [2, 2, 3, 4, ">=", 1],
                  [5, 3, 4, 4, ">=", 1],
                  [3, 2, 1, 6, ">=", 1]],
    max_min="min"
)
lab2_task6_2.solve()

RESULT:
[0.022, 0, 0.1429, 0.1319]
f= 0.29679999999999995


## Задание 8

In [ ]:
lab2_task8_1 = LinearProgrammingTask(
    f=[1, 1, 0],
    restrictions=[[7, 1, ">=", 1],
                  [2, 11, ">=", 1]],
    max_min="min"
)
lab2_task8_1.solve()

RESULT:
[0.1333, 0.0667]
f= 0.2


In [ ]:
lab2_task8_1 = LinearProgrammingTask(
    f=[1, 1, 0],
    restrictions=[[7, 2, "<=", 1],
                  [1, 11, "<=", 1]],
    max_min="max"
)
lab2_task8_1.solve()

RESULT:
[0.12, 0.08]
f= 0.2


## Задание 9

In [ ]:
lab2_task9_1 = LinearProgrammingTask(
    f=[1, 1, 1, 0],
    restrictions=[[7, 2, 3, "<=", 1],
                  [5, 3, 1, "<=", 1],
                  [1, 4, 6, "<=", 1]],
    max_min="max"
)
lab2_task9_1.solve()

RESULT:
[0.0769, 0.1978, 0.022]
f= 0.2967


In [ ]:
lab2_task9_1 = LinearProgrammingTask(
    f=[1, 1, 1, 0],
    restrictions=[[7, 5, 1, ">=", 1],
                  [2, 3, 4, ">=", 1],
                  [3, 1, 6, ">=", 1]],
    max_min="min"
)
lab2_task9_1.solve()

RESULT:
[0.022, 0.1429, 0.1319]
f= 0.29679999999999995


In [ ]:
lab2_task8_1 = LinearProgrammingTask(
    f=[1, 1, 0],
    restrictions=[[0, 1, ">=", 1],
                  [0.5, 0, ">=", 1]],
    max_min="min"
)
lab2_task8_1.solve()

RESULT:
[2.0, 1.0]
f= 3.0
